# csv 파일 데이터를 DB로 넣는 코드

In [4]:
import scipy.io
import csv
import pymysql
from dotenv import load_dotenv
import os
from datetime import datetime
import pandas as pd
import re

## 1. Game 데이터 Insert

In [5]:
# 로컬 DB 커넥팅

load_dotenv()
def get_db(): 
    conn=pymysql.connect(
        host=os.getenv("HOST"),
        port=int(os.getenv("PORT")),
        user=os.getenv("USER"),
        password=os.getenv("PASSWORD"),
        db=os.getenv("DB"),
        charset='utf8',
        )
    return conn


In [3]:
# csv 파일 열기
f=open(r'csv_list/total_games.csv')
csvReader=csv.reader(f)
curs = conn.cursor()

for row in csvReader :
    name = (row[0])
    date = (row[1])
    
    if name=='name' or date=='date' :
        continue

    # dateFormat = '%Y-%m-%d'
    # date = datetime.strptime(date,dateFormat)
    
    sql="insert into home_game (name,release_date) values (%s,%s)"
    curs.execute(sql,(name,date))



In [4]:
# DB 커밋
conn.commit()
f.close()
conn.close()

## Function csv2db (DB data insertion 자동화)

In [29]:
def csv2db(csv_path_list): 
    
    for csv_path in csv_path_list :
        conn = get_db()
        curs=conn.cursor() 
     
        csv_path = 'csv_list/' + csv_path 
        _, table, _= re.split('[/.]',csv_path)
    
        df = pd.read_csv(csv_path)

        try :
            df.drop('id', axis=1, inplace=True)
        except :
            pass
        try :
            df['game_id']=df['game_id'].apply(lambda x:x+1)
        except :
            pass
        try :
            df['tag_id']=df['tag_id'].apply(lambda x:x+1)
        except :
            pass
        try :
            df['publisher_id']=df['publisher_id'].apply(lambda x:x+1)
        except :
            pass
        try :
            df['publisher_game_id']=df['publisher_game_id'].apply(lambda x:x+1)
        except :
            pass
        try :
            df['stream_id']=df['stream_id'].apply(lambda x:x+1)
        except :
            pass
        
        sql_col = ''
        sql_values = ''
        for key in df.keys():
            sql_col += f'{key},'
            sql_values += '%s,'
        sql_col, sql_values = sql_col.rstrip(','), sql_values.rstrip(',')
        sql = f"insert into {table} ({sql_col}) values ({sql_values})"
        
        values_list = []
        for v in df.values :
            values_list.append(tuple(v))

        
        curs.execute('set FOREIGN_KEY_CHECKS = 0')
        curs.execute(f'truncate table {table}')
        print(table)
        
        curs.executemany(sql, values_list)
        conn.commit()
        curs.close()
        conn.close()


# def csv2db(*csv_paths):
#     curs=conn.cursor()
#     conn.commit()   
#     for csv_path in csv_paths :
#         table = csv_path.lstrip('csv_list/').rstrip('.csv')
#         with open(csv_path,'r') as f:
#             csvReader=csv.reader(f)
#             isColumns = True
#             columns_num = 0
#             sql_cols = ''
#             sql_values = ''
#             for row in csvReader :
#                 if isColumns : 
#                     columns_num = len(row)
#                     for idx in range(columns_num):
#                         sql_cols += f'{(row[idx])},'
#                         sql_values += '%s,'

#                     isColumns = False
#                 else : 
#                     value_list = ()
#                     for idx in range(columns_num):
#                         value_list += ((row[idx]),)

#                     sql_cols, sql_values = sql_cols.rstrip(','), sql_values.rstrip(',')
#                     sql = f"insert into {table} ({sql_cols}) values ({sql_values})"
#                     curs.execute(sql,value_list)
#             conn.commit()
#             conn.close()

        

In [30]:
csv_path_list = os.listdir('csv_list')

csv2db(csv_path_list)

home_game
home_gamecomment
home_game_tags
home_publisher
home_publishergame
home_streamgame
home_streamsite
home_tag


In [26]:
df = pd.read_csv('csv_list/home_gamecomment.csv')

df.dropna(inplace=True)

df.to_csv('csv_list/home_gamecomment.csv', index=False)

In [26]:
path_list = []
for csv in os.listdir('csv_list'):
    path_list.append('csv_list/'+csv)

for path_list in 

['csv_list/home_game.csv',
 'csv_list/home_gamecomment.csv',
 'csv_list/home_game_tags.csv',
 'csv_list/home_publisher.csv',
 'csv_list/home_publishergame.csv',
 'csv_list/home_streamgame.csv',
 'csv_list/home_streamsite.csv',
 'csv_list/home_tag.csv']